# Soribada 실시간

## 라이브러리 로드

In [6]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from datetime import datetime
from selenium import webdriver
import chromedriver_autoinstaller as ca

## 준비

### 현재 경로 확인

In [53]:
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/music'

### 폴더 생성

In [55]:
# 파일 저장할 폴더 생성
crawled_folder_path = code_path + '/crawled_data/live_soribada/'
os.makedirs(crawled_folder_path, exist_ok=True)

### 크롬 드라이브 버전 확인

In [10]:
# 현재 크롬 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'100'

In [11]:
# 크롬 드라이버 확인 및 설치
try:
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')
except:
    ca.install(True)
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5868/231581722.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')


## 페이지 접속

In [12]:
# 페이지 접속
url = 'https://www.soribada.com/music/chart'
driver.get(url)
driver.implicitly_wait(3)
driver.maximize_window()

### 수프 담기

In [ ]:
# 수프 담기
soup = bs(driver.page_source, 'lxml')
soup.title.text

In [14]:
# 리스트
li_soup = soup.find_all('li', 'listen')
len(li_soup)

100

### 날짜

In [20]:
# 날짜
rank_date = soup.find('span', 'nowText').text
rank_date

'2022년 04월 04일 07시 10분'

In [21]:
rank_date = rank_date.replace('년 ', '-')
rank_date = rank_date.replace('월 ', '-')
rank_date = rank_date.replace('일 ', '_')
rank_date = rank_date.replace('시 ', ':')
rank_date = rank_date.replace('분', '')
rank_date

'2022-04-04_07:10'

In [22]:
# 날짜2
repeat_rank_date_list = []
for i in range(100):
    repeat_rank_date_list.append(rank_date)
len(repeat_rank_date_list)

100

### 순위

In [46]:
# rank
rank_list = []
for i in range(1, 101):
    rank_list.append(i)
len(rank_list)

100

### 노래 제목

In [23]:
# song_title
li_soup[0].find('span', 'song-title').text

'고맙소'

In [24]:
song_title_list = []
for one in li_soup:
    song_title = one.find('span', 'song-title').text
    song_title_list.append(song_title)
len(song_title_list)

100

In [25]:
song_title_list[:5]

['고맙소', '나보다 더 사랑해요', '아무도 잠들지 마라 (Nessun Dorma)', '만개 (Prod. 신지후)', '할무니']

### 가수

In [28]:
li_soup[0].find('span', 'link-type2-name artist').find('a').text.strip()

'김호중'

In [30]:
li_soup[49].find('span', 'link-type2-name artist').find('a').text.strip()

'야긴'

In [32]:
for i, one in enumerate(li_soup):
    try:
        artist = one.find('span', 'link-type2-name artist').find('a').text.strip()
    except:
        print(i)

27
34
42
43
51
55
56
87
93


In [34]:
li_soup[34].find('span', 'link-type2-name artist detail_artist').text.strip()

'김호중, 오현우'

In [ ]:
for i, one in enumerate(li_soup):
    try:
        artist = one.find('span', 'link-type2-name artist').find('a').text.strip()
    except:
        artist = one.find('span', 'link-type2-name artist detail_artist').text.strip()

In [35]:
# artist
artist_list = []
for one in li_soup:
    try:
        artist = one.find('span', 'link-type2-name artist').find('a').text.strip()
    except:
        artist = one.find('span', 'link-type2-name artist detail_artist').text.strip()
    artist_list.append(artist)
len(artist_list)

100

In [37]:
artist_list[27]

'진시몬, 김호중'

### 앨범

In [38]:
li_soup[0].find('span', 'link-type2 album-title').text

'내일은 미스터트롯 결승전 베스트'

In [42]:
for i, one in enumerate(li_soup):
    try:
        album = one.find('span', 'link-type2 album-title').text
    except:
        print(i)

In [43]:
# album
album_list = []
for one in li_soup:
    album = one.find('span', 'link-type2 album-title').text
    album_list.append(album)
len(album_list)

100

In [44]:
album_list[27]

'<불후의 명곡 – 전설을 노래하다> - 여름특집 1탄-친구 특집'

## df

In [47]:
# df
dict = {'날짜':repeat_rank_date_list, '순위':rank_list, '곡':song_title_list, '가수':artist_list, '앨범':album_list}
df = pd.DataFrame(dict)

In [49]:
df.head()

,날짜,순위,곡,가수,앨범
0,2022-04-04_07:10,1,고맙소,김호중,내일은 미스터트롯 결승전 베스트
1,2022-04-04_07:10,2,나보다 더 사랑해요,김호중,나보다 더 사랑해요
2,2022-04-04_07:10,3,아무도 잠들지 마라 (Nessun Dorma),김호중,The Classic Album
3,2022-04-04_07:10,4,만개 (Prod. 신지후),김호중,우리家
4,2022-04-04_07:10,5,할무니,김호중,할무니


In [57]:
date = rank_date.split('_')[0].replace('-', '')
date

'20220404'

## 파일 만들기

In [58]:
# make excel
today_date = datetime.today().strftime("%Y%m%d_%H%M%S")
date = rank_date.split('_')[0].replace('-', '')
file_name = f'live_soribada_{date}_{today_date}.xlsx'
df.to_excel(crawled_folder_path + file_name, index=False)

print(f"{file_name} 파일 생성 완료")

live_soribada_20220404_20220422_114105.xlsx 파일 생성 완료


# 코드 마지막